In [1]:
using QuantEcon
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [2]:
@everywhere mutable struct RBC{T <: Real}
    α::Float64
    β::Float64
    z::Vector{T}
    T::Matrix{T}
    K::Float64
    out::Float64
    cs::Float64
    k_grid::Vector{T}
    output::Matrix{T}
    f_value::Matrix{Float64}
    n::Int64
    m::Int64

end


In [3]:
@everywhere function RBC(;α = 1/3,
                          β = 0.95)
    
    mc = QuantEcon.tauchen(100, 0.9, 0.1, 0.5)
    z =  mc.state_values
    T = mc.p
    K = (α * β)^(1 / (1 - α))
    out = K^α
    cs = out - K
    k_grid = collect(0.5*K:0.00001:1.5*K)
    output = (k_grid.^α) * z';
    n = length(k_grid)
    m = length(z)
    f_value  = zeros(n, m)
    RBC(α, β, z, T, K, out, cs, k_grid, output, f_value, n, m)
    
end



In [4]:
function vfi!(rbc::RBC,
              bellman_operator!::Function; tol::Float64=1e-7,error::Float64=10.0)
    iter = 0
    fpolicy = 0
    while error > tol 
        (error, fpolicy) = bellman_operator!(rbc)
        iter = iter + 1
        if mod(iter, 10)==0 || iter == 1
            println(" Iteration = ", iter, " Sup Diff = ", error)
        end 
    end
    
    println(" My check = ", fpolicy)

end

vfi! (generic function with 1 method)

In [5]:
@everywhere function bellman_operator!(rbc::RBC)

        f_valueN = SharedArray{Float64,2}(rbc.n, rbc.m)
        f_policy = SharedArray{Float64,2}(rbc.n, rbc.m)
        E = rbc.f_value * rbc.T'
        @sync @parallel for i = 1: rbc.m
            kNext = 1
            for j = 1:rbc.n
                v_max = -1000.0
                k_choice  = rbc.k_grid[1]
                    for l = kNext : rbc.n
                        c = rbc.output[j, i] - rbc.k_grid[l]
                        v = (1 - rbc.β) * log(c) + rbc.β * E[l, i]
                        if v > v_max
                            v_max = v
                            k_choice = rbc.k_grid[l]
                            kNext = l
                        else
                            break 
                        end
                                 
                    end
                    f_valueN[j, i] = v_max
                    f_policy[j, i] = k_choice
              end
        end    
    error  = maximum(abs.(f_valueN - rbc.f_value))
    rbc.f_value = f_valueN
    return (error, f_policy[1000, 3])
end


In [6]:
rbc = RBC();

In [7]:
@time vfi!(rbc, bellman_operator!)

 Iteration = 1 Sup Diff = 0.06369885100730058
 Iteration = 10 Sup Diff = 0.03569333797489649
 Iteration = 20 Sup Diff = 0.02070733980397277
 Iteration = 30 Sup Diff = 0.012259080086808094
 Iteration = 40 Sup Diff = 0.007311075640998421
 Iteration = 50 Sup Diff = 0.004371439831109214
 Iteration = 60 Sup Diff = 0.0026161100113157865
 Iteration = 70 Sup Diff = 0.001566107423631724
 Iteration = 80 Sup Diff = 0.0009376339127058486
 Iteration = 90 Sup Diff = 0.0005613852395218721
 Iteration = 100 Sup Diff = 0.000336119848320493
 Iteration = 110 Sup Diff = 0.0002012469088676916
 Iteration = 120 Sup Diff = 0.00012049386329548994
 Iteration = 130 Sup Diff = 7.214410732059306e-5
 Iteration = 140 Sup Diff = 4.319533796159902e-5
 Iteration = 150 Sup Diff = 2.5862643607155178e-5
 Iteration = 160 Sup Diff = 1.548491990233103e-5
 Iteration = 170 Sup Diff = 9.271393511145476e-6
 Iteration = 180 Sup Diff = 5.551125766167786e-6
 Iteration = 190 Sup Diff = 3.323664049315056e-6
 Iteration = 200 Sup Diff =

In [8]:
@time vfi!(rbc, bellman_operator!)

 Iteration = 1 Sup Diff = 9.167892134342992e-8
 My check = 0.2672891436962635
  0.146118 seconds (2.84 k allocations: 54.485 MiB, 4.56% gc time)
